In [71]:
import numpy as np

class MLP:
    
    def __init__(self):
        self.layers = list()
    
    def addLayers(self, layers):
        for layer in layers:
            self.layers.append(layer)
        
    def train(self, l_rate = 0.25, iters = 101):
        n_layers = len(self.layers)
        
        for i in range(iters):
            self.forward()
            error = 0.5 * np.sum((self.layers[n_layers - 1].value - self.layers[n_layers - 2].value)**2)
            
            if (i%100 == 0):
                print("Error: {:.3f}".format(error))
            
            # back-propogation
            index = n_layers - 3
            
            # calculate etah_0 for last hidden layer
            # etah = -(y - y^) * (1 - y^) * y^
            self.layers[index].weight.etah = -(self.layers[n_layers - 1].value - self.layers[index+1].value) * \
                                        (1 - self.layers[index+1].value) * self.layers[index+1].value
            index -= 1
                                        
            # calculate etah
            while (index >= 0):
                # etah_i-1 = <etah_i, (<G_i, W_i>)^T> * (1 - g_i) * g_i
                self.layers[index].weight.etah = np.dot(self.layers[index+1].weight.etah,
                                                np.dot(self.layers[index+1].G, self.layers[index+1].weight.value).T) * \
                                                (1 - self.layers[index+1].value_before_G) * \
                                                self.layers[index+1].value_before_G
                index -= 1
            
            # update weights using etah, going downhill
            index = 0
            while (index < n_layers - 2):
                self.layers[index].weight.value -= l_rate * np.dot(self.layers[index].value_after_G.T,
                                                            self.layers[index].weight.etah)
                index += 1
        
    
    def forward(self, bias = -1):
        # calculate values for each layer
        # len - 1 is for excluding the target layer
        # len - 2 is the output layer
        # 0 is the input layer
        
        index = 0
        while (index < len(self.layers)-1):
            if (index == 0):
                self.layers[index].value_before_G = self.layers[index].value
            else:
                self.layers[index].value_before_G = self.sigmoid(np.dot(self.layers[index-1].value_after_G,
                                        self.layers[index-1].weight.value))                
            
            if (index == len(self.layers)-2):
                self.layers[index].value = self.layers[index].value_before_G
                self.layers[index].value_after_G = self.layers[index].value_before_G
            else:
                self.layers[index].value_after_G = np.concatenate((self.layers[index].value_before_G,
                                     np.full(shape = (self.layers[index].value_before_G.shape[0], 1),
                                            fill_value = bias)), axis = 1)
                
                # product_bias = value_before_G * G
                # G = inv(value_before_G) * product_bias
                self.layers[index].G = np.dot(np.linalg.pinv(self.layers[index].value_before_G),
                                          self.layers[index].value_after_G)
            index += 1
    
            
    def predict(self, x, bias = -1):
        x = np.concatenate((x,np.full(shape = (x.shape[0], 1),
                                            fill_value = bias)), axis = 1)
        index = 1
        output = x
        while (index < len(self.layers)-1):
            output = self.sigmoid(np.dot(output, self.layers[index-1].weight.value))
            
            if (index != len(self.layers)-2):
                output = np.concatenate((output, np.full(shape = (output.shape[0], 1),
                                            fill_value = bias)), axis = 1)
            index += 1
        return output
            
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    

class Layer:
    
    def __init__(self, rows = None, cols = None, value = None):
        self.rows = rows
        self.cols = cols
        self.value = value
        if (value is not None):
            self.rows = self.value.shape[0]
            self.cols = self.value.shape[1]
        self.value_before_G = None
        self.value_after_G = None
        self.G = None
        self.weight = None
        
    def connect(self, layer):
        self.weight = Weight(value = np.random.normal(scale=1 / (self.cols+1)**.5, 
                                                      size=(self.cols+1, layer.cols)))

class Weight:
    
    def __init__(self, value):
        self.etah = None
        self.value = value

In [72]:
mlp = MLP()
x = np.linspace(0,1,40).reshape((40,1))
x = (x-0.5)*2
y = np.sin(2*np.pi*x) + np.cos(4*np.pi*x) + np.random.randn(40).reshape((40,1))*0.2


train = x[0::2,:]
test = x[1::4,:]
valid = x[3::4,:]
traintarget = y[0::2,:]
testtarget = y[1::4,:]
validtarget = y[3::4,:]

X = Layer(value = train)
target = Layer(value = traintarget)

h1 = Layer(rows = X.rows, cols = 3)
h2 = Layer(rows = X.rows, cols = 2)
h3 = Layer(rows = X.rows, cols = 1)
output = Layer(rows = X.rows, cols = target.cols)

X.connect(h1)
h1.connect(h2)
h2.connect(h3)
h3.connect(output)

mlp.addLayers([X, h1,
               h2,
               h3,
               output, target])

mlp.train()

Error: 11.011
Error: 9.623


In [74]:
test_x = np.ones((1,1))
output = mlp.predict(x=test_x)

print(output)

[[0.03318409]]
